### 다음 위치에서 판례정리.zip을 다운받아서 같은 폴더 안에서 압축 풀기

https://drive.google.com/file/d/1zRF8tq9DQrMrXJBirvWZh6QAm5j7r9FN/view?usp=sharing

### 다음 코드를 참고하여 Elasticsearch에 nori tokenizer 설치

```
docker exec -it elasticsearch-es01-1 /usr/share/elasticsearch/bin/elasticsearch-plugin install analysis-nori
docker exec -it elasticsearch-es02-1 /usr/share/elasticsearch/bin/elasticsearch-plugin install analysis-nori
docker exec -it elasticsearch-es03-1 /usr/share/elasticsearch/bin/elasticsearch-plugin install analysis-nori

docker restart elasticsearch-es01-1 elasticsearch-es02-1 elasticsearch-es03-1

docker exec -it elasticsearch-es01-1 /usr/share/elasticsearch/bin/elasticsearch-plugin list
```

In [1]:
import os, json
from langchain_core.documents import Document
from langchain_elasticsearch import ElasticsearchStore
from elasticsearch import Elasticsearch
from langchain_elasticsearch import DenseVectorStrategy

from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="nlpai-lab/KURE-v1",
    model_kwargs={"device": 'cpu'}
)

# ===== 1. Elasticsearch Client 생성 (SSL 검증 끔) =====
es_client = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic", "changeme"),
    verify_certs=False,
    ssl_show_warn=False
)

# ===== 2. Index & Embedding 객체 준비 =====
index_name = "cases"

elastic_vector_search = ElasticsearchStore(
    es_connection=es_client,
    index_name=index_name,
    embedding=embeddings,
)

# ===== 3. JSON 파일 로드 =====
folder = "판례정리"
files = [f for f in os.listdir(folder) if f.endswith(".json")]

def load_cases():
    all_cases = []
    for file in files:
        path = os.path.join(folder, file)
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)

        # 파일 하나에 여러 판례가 들어있는 형태라 가정
        for case in data:
            text = case.get("판례내용", "")
            metadata = {k: v for k, v in case.items() if k != "판례내용"}

            doc = Document(page_content=text, metadata=metadata)
            all_cases.append(doc)

    return all_cases

docs = load_cases()
print(f"총 문서 개수: {len(docs)}")


/var/folders/fn/cyp1s2wj7c39s35jrjsksnkw0000gn/T/ipykernel_98906/913043851.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/Users/oksjjj/coding/mylawyer/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


총 문서 개수: 29953


In [2]:
start_inserting = False
batch_size = 100

for i in range(0, len(docs), batch_size):
    batch = docs[i:i+batch_size]
    filtered_batch = []

    for doc in batch:
        case_id = doc.metadata.get("판례일련번호")

        # 이미 삽입 시작 → 체크 생략
        if start_inserting:
            filtered_batch.append(doc)
            continue

        # 인덱스가 없는 경우 (처음 실행 시)
        if not es_client.indices.exists(index=index_name):
            start_inserting = True
            filtered_batch.append(doc)
            continue

        # 존재 확인 쿼리
        res = es_client.search(
            index=index_name,
            body={
                "query": {
                    "term": {
                        "metadata.판례일련번호": case_id
                    }
                }
            }
        )

        # 기존 있음 → skip
        if res["hits"]["total"]["value"] > 0:
            continue

        # 나머지 → 삽입 시작
        start_inserting = True
        filtered_batch.append(doc)


    # ===== 출력부 수정 =====
    if len(filtered_batch) == 0:
        print(f"{i} ~ {i + batch_size - 1} 구간: 신규 문서 없음 (총 0)")
        print("→ 신규 데이터 없음, SKIP")
        continue
    else:
        print(f"업로드 중: {i} ~ {i + len(filtered_batch) - 1} (총 {len(filtered_batch)})")


    # ===== 문서 삽입 =====
    ElasticsearchStore.from_documents(
        documents=filtered_batch,
        embedding=embeddings,
        es_connection=es_client,
        index_name=index_name,
    )

0 ~ 99 구간: 신규 문서 없음 (총 0)
→ 신규 데이터 없음, SKIP
100 ~ 199 구간: 신규 문서 없음 (총 0)
→ 신규 데이터 없음, SKIP
200 ~ 299 구간: 신규 문서 없음 (총 0)
→ 신규 데이터 없음, SKIP
300 ~ 399 구간: 신규 문서 없음 (총 0)
→ 신규 데이터 없음, SKIP
400 ~ 499 구간: 신규 문서 없음 (총 0)
→ 신규 데이터 없음, SKIP
500 ~ 599 구간: 신규 문서 없음 (총 0)
→ 신규 데이터 없음, SKIP
600 ~ 699 구간: 신규 문서 없음 (총 0)
→ 신규 데이터 없음, SKIP
700 ~ 799 구간: 신규 문서 없음 (총 0)
→ 신규 데이터 없음, SKIP
800 ~ 899 구간: 신규 문서 없음 (총 0)
→ 신규 데이터 없음, SKIP
900 ~ 999 구간: 신규 문서 없음 (총 0)
→ 신규 데이터 없음, SKIP
1000 ~ 1099 구간: 신규 문서 없음 (총 0)
→ 신규 데이터 없음, SKIP
1100 ~ 1199 구간: 신규 문서 없음 (총 0)
→ 신규 데이터 없음, SKIP
1200 ~ 1299 구간: 신규 문서 없음 (총 0)
→ 신규 데이터 없음, SKIP
1300 ~ 1399 구간: 신규 문서 없음 (총 0)
→ 신규 데이터 없음, SKIP
1400 ~ 1499 구간: 신규 문서 없음 (총 0)
→ 신규 데이터 없음, SKIP
1500 ~ 1599 구간: 신규 문서 없음 (총 0)
→ 신규 데이터 없음, SKIP
1600 ~ 1699 구간: 신규 문서 없음 (총 0)
→ 신규 데이터 없음, SKIP
1700 ~ 1799 구간: 신규 문서 없음 (총 0)
→ 신규 데이터 없음, SKIP
1800 ~ 1899 구간: 신규 문서 없음 (총 0)
→ 신규 데이터 없음, SKIP
1900 ~ 1999 구간: 신규 문서 없음 (총 0)
→ 신규 데이터 없음, SKIP
2000 ~ 2099 구간: 신규 문서 없음 (총 0)
→ 신규 데이터 없음,

In [3]:
def rrf_fusion(bm25_hits, knn_hits, k=60):
    scores = {}
    methods = {}  # 문서별 최종 출처 저장 (Sparse / Dense)

    # Sparse (BM25)
    for rank, (doc_id, _, method) in enumerate(bm25_hits, start=1):
        scores[doc_id] = scores.get(doc_id, 0) + 1 / (k + rank)
        # Sparse로 기록 (나중에 Dense가 덮을 수 있음)
        methods[doc_id] = method

    # Dense (KNN)
    for rank, (doc_id, _, method) in enumerate(knn_hits, start=1):
        scores[doc_id] = scores.get(doc_id, 0) + 1 / (k + rank)
        # Dense가 더 중요하다고 가정 → 동일 문서면 Dense로 덮음
        methods[doc_id] = method

    # 점수 기준 정렬
    ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)

    # (id, score, method) 형태로 반환
    return [(doc_id, score, methods.get(doc_id)) for doc_id, score in ranked]

In [4]:
def format_results(es_client, index_name, ranked_results, top_k=5):
    formatted = []

    for rank, (doc_id, score, method) in enumerate(ranked_results[:top_k], start=1):

        # 문서 조회 (vector 제외)
        doc = es_client.get(
            index=index_name,
            id=doc_id,
            _source_excludes=["vector"]
        )

        src = doc["_source"]

        formatted.append({
            "rank": rank,
            "score": score,
            "method": method,   # Sparse or Dense
            "id": doc_id,
            "text": src.get("text", None),
            "metadata": src.get("metadata", {})
        })

    return formatted

In [15]:
query = "교사가 학생을 훈계하는 과정에서 폭행이 문제된 판례 찾아줘"

vector = embeddings.embed_query(query)

N_DOCS = 5
K = 60

bm25 = es_client.search(
    index=index_name,
    body={"query": {"match": {"text": query}}, "size": N_DOCS},
    _source_excludes=["vector"]
)["hits"]["hits"]

knn = es_client.search(
    index=index_name,
    body={
        "knn": {
            "field": "vector",
            "query_vector": vector,
            "k": N_DOCS,
            "num_candidates": N_DOCS * 10
        }
    },
    _source_excludes=["vector"]
)["hits"]["hits"]

# 방법 라벨링 적용
bm25_labeled = [(hit["_id"], hit["_score"], "Sparse") for hit in bm25]
knn_labeled = [(hit["_id"], hit["_score"], "Dense") for hit in knn]

# 🔥 RRF 수행할 때 labeled 리스트를 넣어야 함 (기존 코드 오류 수정)
results = rrf_fusion(bm25_labeled, knn_labeled, k=K)

# 문서 상세 조회
final_docs = format_results(es_client, index_name, results, top_k=5)

# ===================== 🔍 출력 ===================== #

for d in final_docs:
    print(f"[{d['rank']}위] score={d['score']:.4f} / method={d['method']}")
    print(f"판례ID: {d['metadata'].get('판례일련번호')}")
    print(f"사건명: {d['metadata'].get('사건명')}")
    print(f"선고일자: {d['metadata'].get('선고일자')}")
    print(f"참조조문: {d['metadata'].get('참조조문')}")
    print("\n--- 판례 내용 ---")
    print(d.get("text", "").strip())
    print("-"*80)

[1위] score=0.0323 / method=Dense
판례ID: 83309
사건명: 폭행·모욕
선고일자: 2004.06.10
참조조문: [1] 형법 제20조 / [2] 형법 제20조, 초·중등교육법 제18조 제1항, 제20조 제3항, 초·중등교육법시행령 제31조 제1항, 제7항 / [3] 형법 제20조

--- 판례 내용 ---
【피 고 인】 피고인 【상 고 인】 피고인 【원심판결】 대전지법 2001. 9. 20. 선고 2000노1669 판결 【주 문】 상고를 기각한다. 【이 유】 1. 상고이유 제1주장에 관하여 원심이 인용한 제1심 채용 증거들과 대조하여 보니, 여자중학교 체육교사 겸 태권도 지도교사인 피고인이 교실 밖 공개된 장소에서 피해자 공소외 1, 공소외 2를 폭행하였고 피해자 공소외 3, 공소외 4, 공소외 1에게 욕설을 하여 모욕하였다는 요지의 이 사건 공소사실들이 유죄로 증명되었다고 보아 제1심판결을 유지한 원심의 판단은 옳고 그 판단에 필요한 심리를 다하지 아니하였다거나 증거법칙에 위반하였다는 등의 잘못이 없다. 피고인이 그 공소사실과 같은 행위를 한 적이 없음에도 원심이 증거판단과 사실인정을 잘못하였다는 취지의 이 부분 주장들을 받아들이지 아니한다. 2. 상고이유 제2주장에 관하여 형법 제20조가 법령에 의한 행위 또는 업무로 인한 행위 기타 사회상규에 위반되지 아니하는 행위는 벌하지 아니한다고 규정하여 법령에 의한 학생에 대한 징계나 학생에 대한 교육적 지도행위의 경우에는 그 행위의 위법성이 조각(阻却)되는 것임은 상고이유로 주장된 바와 같다. 그런데 사회상규에 위반되지 아니하는 행위라 함은 법질서 전체의 정신이나 그의 배후에 놓여 있는 사회윤리 도의적 감정 내지 사회통념에 비추어 용인될 수 있는 행위를 말하는 것이어서 어떠한 행위가 사회상규에 위배되지 아니하는가는 구체적 사정 아래에서 합목적적, 합리적으로 고찰하여 개별적으로 판단되어야 할 것이다(대법원 2000. 4. 25. 선고 98도2389 판결 참조). 한편, 교육에 관한 중심